# Test inference speed

## Print versions

In [1]:
# import numpy as np
# import torch

# print(f"{np.__version__=}")
# print(f"{torch.__version__=}")
# print(f"{torch.cuda.is_available()=}")

## Imports

In [2]:
from types import SimpleNamespace

from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


## Config

In [3]:
config = SimpleNamespace(
    model_dir="./debug_ttt/00576224",
    inference_batch_size=1,
)

## Run model

In [9]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=config.model_dir,
    max_seq_length=6144,
    load_in_4bit=False,
    load_in_8bit=False,
)

==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.568 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [11]:
type(model), type(tokenizer)

(transformers.models.llama.modeling_llama.LlamaForCausalLM,
 transformers.tokenization_utils_fast.PreTrainedTokenizerFast)

## LlamaForCausalLM
- [LlamaForCausalLM.forward()](https://github.com/huggingface/transformers/blob/e71a01a104dd663c730e494eb0b6467bb51df357/src/transformers/models/llama/modeling_llama.py#L1137)

The official docs are

- `input_ids` (torch.LongTensor of shape (batch_size, sequence_length)) — Indices of input sequence tokens in the vocabulary. Padding will be ignored by default should you provide it.
Indices can be obtained using AutoTokenizer. See PreTrainedTokenizer.encode() and PreTrainedTokenizer.call() for details.

- `attention_mask` (torch.Tensor of shape (batch_size, sequence_length), optional) — Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]:
- `1` for tokens that are not masked,
- `0` for tokens that are masked.

Indices can be obtained using AutoTokenizer. See `PreTrainedTokenizer.encode()` and `PreTrainedTokenizer.call()` for details.

If `past_key_values` is used, optionally only the last input_ids have to be input (see `past_key_values`).

If you want to change padding behavior, you should read modeling_opt._prepare_decoder_attention_mask and modify to your needs. See diagram 1 in the paper for more information on the default strategy.

1 indicates the head is not masked,
0 indicates the head is masked.

- `position_ids` (torch.LongTensor of shape (batch_size, sequence_length), optional) — Indices of positions of each input sequence tokens in the position embeddings. Selected in the range [0, config.n_positions - 1].

Contrary to RNNs that have the position of each token embedded within them, transformers are unaware of the position of each token. Therefore, the position IDs (position_ids) are used by the model to identify each token’s position in the list of tokens.

- `past_key_values` (Cache or tuple(tuple(torch.FloatTensor)), optional) — Pre-computed hidden-states (key and values in the self-attention blocks and in the cross-attention blocks) that can be used to speed up sequential decoding. This typically consists in the past_key_values returned by the model at a previous stage of decoding, when `use_cache=True` or `config.use_cache=True`.

Two formats are allowed:

  - a Cache instance, see our [kv cache guide](https://huggingface.co/docs/transformers/en/kv_cache);
  - Tuple of tuple(torch.FloatTensor) of length config.n_layers, with each tuple having 2 tensors of shape (batch_size, num_heads, sequence_length, embed_size_per_head)). This is also known as the legacy cache format.

The model will output the same cache format that is fed as input. If no past_key_values are passed, the legacy cache format will be returned.

If past_key_values are used, the user can optionally input only the last input_ids (those that don’t have their past key value states given to this model) of shape (batch_size, 1) instead of all input_ids of shape (batch_size, sequence_length).

- `inputs_embeds` (torch.FloatTensor of shape (batch_size, sequence_length, hidden_size), optional) — Optionally, instead of passing input_ids you can choose to directly pass an embedded representation. This is useful if you want more control over how to convert input_ids indices into associated vectors than the model’s internal embedding lookup matrix.

- `use_cache` (bool, optional) — If set to True, past_key_values key value states are returned and can be used to speed up decoding (see past_key_values).

- `output_attentions` (bool, optional) — Whether or not to return the attentions tensors of all attention layers. See attentions under returned tensors for more detail.

- `output_hidden_states` (bool, optional) — Whether or not to return the hidden states of all layers. See hidden_states under returned tensors for more detail.

- `return_dict` (bool, optional) — Whether or not to return a ModelOutput instead of a plain tuple.

- `cache_position` (torch.LongTensor of shape (sequence_length), optional) — Indices depicting the position of the input sequence tokens in the sequence. Contrarily to position_ids, this tensor is not affected by padding. It is used to update the cache in the correct position and to infer the complete sequence length.
Args — labels (torch.LongTensor of shape (batch_size, sequence_length), optional): Labels for computing the masked language modeling loss. Indices should either be in [0, ..., config.vocab_size] or -100 (see input_ids docstring). Tokens with indices set to -100 are ignored (masked), the loss is only computed for the tokens with labels in [0, ..., config.vocab_size].

- `num_logits_to_keep` (int, optional): Calculate logits for the last num_logits_to_keep tokens. If 0, calculate logits for all input_ids (special case). Only last token logits are needed for generation, and calculating them only for that token can save memory, which becomes pretty significant for long sequences or large vocabulary size.

**Returns**

- transformers.modeling_outputs.CausalLMOutputWithPast or tuple(torch.FloatTensor)